# 2 DuckDB在Python中的使用

## 2.1 数据集的导入

In [1]:
import numpy as np
import pandas as pd

In [2]:
# 利用pandas生成示例数据文件
generated_df = pd.DataFrame(
    {
        '类别': np.random.choice(list('ABCDEF'), 5000000),
        '数值': np.round(np.random.uniform(0, 1000000, 5000000), 3)
    }
)

# 分别导出为csv、parquet、json格式
generated_df.to_csv('./demo_data.csv', index=False)
generated_df.to_parquet('./demo_data.parquet')
generated_df.to_json('./demo_data.json', index=False)

In [3]:
import duckdb
import polars as pl

- `csv`格式

In [4]:
%%timeit 
duckdb.read_csv('./demo_data.csv')

44.2 ms ± 2.83 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [5]:
%%timeit 
pd.read_csv('./demo_data.csv')

1.68 s ± 90.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
%%timeit 
pl.read_csv('./demo_data.csv')

120 ms ± 3.87 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


- `parquet`格式

In [7]:
%%timeit 
duckdb.read_parquet('./demo_data.parquet')

1.04 ms ± 28.8 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [8]:
%%timeit 
pd.read_parquet('./demo_data.parquet')

589 ms ± 21 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
%%timeit 
pl.read_parquet('./demo_data.parquet')

82.6 ms ± 1.24 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [10]:
duckdb.sql("SELECT * FROM 'demo_data.csv' LIMIT 3")

┌─────────┬────────────┐
│  类别   │    数值    │
│ varchar │   double   │
├─────────┼────────────┤
│ A       │ 921260.353 │
│ C       │  139757.95 │
│ C       │  775391.17 │
└─────────┴────────────┘

In [11]:
duckdb.sql("SELECT * FROM 'demo_data.parquet' LIMIT 3")

┌─────────┬────────────┐
│  类别   │    数值    │
│ varchar │   double   │
├─────────┼────────────┤
│ A       │ 921260.353 │
│ C       │  139757.95 │
│ C       │  775391.17 │
└─────────┴────────────┘

### 2.1.2 读取其他框架的数据对象

In [12]:
pandas_df = pd.read_parquet('demo_data.parquet')
pandas_df.head(1)

,类别,数值
0,A,921260.353


In [13]:
polars_df = pl.read_parquet('demo_data.parquet')
polars_df.head(1)

类别,数值
str,f64
"""A""",921260.353


In [14]:
duckdb.sql("SELECT * FROM pandas_df LIMIT 1")

┌─────────┬────────────┐
│  类别   │    数值    │
│ varchar │   double   │
├─────────┼────────────┤
│ A       │ 921260.353 │
└─────────┴────────────┘

In [15]:
duckdb.sql("SELECT * FROM polars_df LIMIT 1")

┌─────────┬────────────┐
│  类别   │    数值    │
│ varchar │   double   │
├─────────┼────────────┤
│ A       │ 921260.353 │
└─────────┴────────────┘

## 2.2 执行分析运算

In [16]:
demo_r = duckdb.read_parquet('demo_data.parquet')
type(demo_r)

duckdb.duckdb.DuckDBPyRelation

In [17]:
# 查看第一行
duckdb.sql("SELECT * FROM demo_r LIMIT 1")

┌─────────┬────────────┐
│  类别   │    数值    │
│ varchar │   double   │
├─────────┼────────────┤
│ A       │ 921260.353 │
└─────────┴────────────┘

In [18]:
# 统计数据记录数
duckdb.sql("SELECT count(1) FROM demo_r")

┌──────────┐
│ count(1) │
│  int64   │
├──────────┤
│  5000000 │
└──────────┘

In [19]:
# 计算类别字段唯一值数量
duckdb.sql("SELECT count(DISTINCT 类别) FROM demo_r")

┌────────────────────────┐
│ count(DISTINCT "类别") │
│         int64          │
├────────────────────────┤
│                      6 │
└────────────────────────┘

In [20]:
# 分组统计平均值
duckdb.sql("""SELECT 类别, mean(数值) FROM demo_r GROUP BY 类别""")

┌─────────┬────────────────────┐
│  类别   │    mean("数值")    │
│ varchar │       double       │
├─────────┼────────────────────┤
│ C       │  499861.3371859329 │
│ F       │  500054.2954633667 │
│ D       │  500010.8334199005 │
│ A       │  499840.9485479467 │
│ B       │ 500181.61871720897 │
│ E       │  500059.2490139358 │
└─────────┴────────────────────┘

In [21]:
%%timeit
duckdb.sql("""SELECT 类别, mean(数值) FROM demo_r GROUP BY 类别""")

673 µs ± 53.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [22]:
%%timeit
pandas_df.groupby('类别')['数值'].mean()

384 ms ± 65.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [23]:
%%timeit
polars_df.group_by('类别').agg(pl.col('数值').mean())  

56.1 ms ± 3.75 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## 2.3 计算结果转换

In [24]:
# 转为python对象
duckdb.sql("SELECT * FROM demo_r LIMIT 1").fetchall()

[('A', 921260.353)]

In [25]:
# 转为pandas数据框
duckdb.sql("SELECT * FROM demo_r LIMIT 1").df()

,类别,数值
0,A,921260.353


In [26]:
# 转为polars数据框
duckdb.sql("SELECT * FROM demo_r LIMIT 1").pl()

类别,数值
str,f64
"""A""",921260.353


In [27]:
# 转为numpy数组
duckdb.sql("SELECT * FROM demo_r LIMIT 1").fetchnumpy()

{'类别': array(['A'], dtype=object), '数值': array([921260.353])}

- `csv`格式

In [28]:
%%timeit
demo_r.write_csv('csv_output_test.csv')

699 ms ± 18.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [29]:
%%timeit
pandas_df.to_csv('csv_output_test.csv', index=False)

8.19 s ± 90 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [30]:
%%timeit
polars_df.write_csv('csv_output_test.csv')

185 ms ± 12.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


- `parquet`格式

In [31]:
%%timeit
demo_r.write_parquet('csv_output_test.parquet')

411 ms ± 18.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [32]:
%%timeit
pandas_df.to_parquet('csv_output_test.parquet')

1.59 s ± 47.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [33]:
%%timeit
polars_df.write_parquet('csv_output_test.parquet')

989 ms ± 36.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
